In [1]:
import pandas as pd
import numpy as np
from tkinter import filedialog as fd

In [2]:
# Import the raw data
# is_select = input('Would you like to search for a file (Y\\N): ')
# if(is_select == 'Y'):
#     filename = fd.askopenfilename()
#     raw_patron_df = pd.read_excel(filename)
# else:
#     raw_patron_df = pd.read_excel('C:\\Users\\Ben\\Desktop\\HRP\\New Project\\Data\\Raw_Data\\Patron_Checkouts.xlsx')

raw_patron_df = pd.read_excel('C:\\Users\\Ben\\Desktop\\HRP\\New Project\\Data\\Raw_Data\\Patron_Checkouts.xlsx')

In [3]:
raw_patron_df.sample(5)

,Patron Barcode,Title,Author's Name,Holdings Barcode,Date of Action,Unnamed: 5,Unnamed: 6
27315,13148,COMPUTER 4,NaN,510004,11/16/2021 04:11:29PM,NaN,NaN
61977,14012,The Hunger Games: Catching Fire,NaN,802511,07/23/2022 01:22:05PM,NaN,NaN
74851,17366,Moo who?,"Palatini, Margie",128007,07/02/2021 01:58:53PM,NaN,NaN
3885,11385,The immortal life of Henrietta Lacks,"Skloot, Rebecca,",90000615,02/09/2021 02:47:22PM,NaN,NaN
41932,13422,Tom and Jerry: the movie,NaN,801708,08/23/2021 01:58:44PM,NaN,NaN


In [4]:
raw_patron_df.shape

(90878, 7)

# Code

## Adjustments

In [5]:
# Set to true to keep the date column
keep_date = False
# Set to true to keep ILL entries
keep_ILL = False
# Set to true to remove all items with na values in the first and last name column
keep_na_authors = False
# Set to true to keep duplicate entries in the patron data
keep_duplicates = True

# Set to true to replace the title and author column in the patron data with the index from inventory
index_patron = True

## Clean Patron Data

In [6]:
# Drop shifted columns
patron_df = raw_patron_df[raw_patron_df['Unnamed: 5'].isnull()]
patron_df = patron_df[patron_df['Unnamed: 6'].isnull()]

patron_df.shape

(90833, 7)

In [7]:
# Rename Patron Barcode
patron_df = patron_df.rename(columns = {'Patron Barcode': 'Patron_ID'})

In [8]:
# Drop unnecessary columns
patron_df = patron_df.drop(['Holdings Barcode', 'Unnamed: 5', 'Unnamed: 6'], axis = 1)

In [9]:
# Drop the date
if(not keep_date):
    patron_df = patron_df.drop('Date of Action', axis = 1)

In [10]:
# Find all ILL entries
patron_df['Is_ILL'] = patron_df['Title'].str.contains('ILL -', na = False)
if(not keep_ILL):
    # Remove all ILL entries
    patron_df = patron_df[patron_df['Is_ILL'] == False]
    patron_df = patron_df.drop('Is_ILL', axis = 1)
    
    print(patron_df.shape)

(89305, 3)


In [11]:
# Split mutiple authors on ;
patron_df[['1', '2', '3', '4', '5', '6']] = patron_df['Author\'s Name'].str.split(';', expand = True)
patron_df = patron_df.drop(['Author\'s Name', '2', '3', '4', '5', '6'], axis = 1)

# Split authors first name and last name on ,
patron_df[['Author_Last', 'Author_First', '1', '2', '3']] = patron_df['1'].str.split(',', expand = True)
patron_df = patron_df.drop(['1', '2', '3'], axis = 1)

In [12]:
# Drop na authors
if(not keep_na_authors):
    # Drop all entries where both author last and first are na
    patron_df = patron_df.dropna(subset = ['Author_Last', 'Author_First'], how = 'all')
    
    print(patron_df.shape)

(75487, 4)


In [13]:
# Drop na titles or patrons
patron_df = patron_df.dropna(subset = ['Patron_ID', 'Title'])

patron_df.shape

(75481, 4)

In [14]:
# Remove punctation and convert to lowercase
cols = ['Title', 'Author_Last', 'Author_First']
for col in cols:
    patron_df[col] = patron_df[col].str.replace('[^\w\s]','', regex = True)
    patron_df[col] = patron_df[col].str.lower()

In [15]:
if(not keep_duplicates):
    patron_df = patron_df.drop_duplicates()
    
    print(patron_df.shape)

## Create an Inventory

In [16]:
# Select title and authors name to create inventory
inventory_df = patron_df[['Title', 'Author_First', 'Author_Last']].copy()

In [17]:
# Count how many times each book has been checked out
inventory_df = inventory_df.value_counts().reset_index()
inventory_df.rename(columns = {0: 'Num_Checkouts'}, inplace = True)

inventory_df

,Title,Author_First,Author_Last,Num_Checkouts
0,sonic the hedgehog,ian,flynn,235
1,babysitters little sister,katy,farina,163
2,if you give a pig a pancake,laura joffe,numeroff,119
3,beastars,paru,itagaki,114
4,if you give a dog a donut,laura joffe,numeroff,112
...,...,...,...,...
11433,watch that witch,nancy e,krulik,1
11434,mastering the art of french cooking,julia,child,1
11435,water,jason,cooper,1
11436,stop whining start living,laura,schlessinger,1


## Index the Patron Data by the Inventory

In [18]:
# Create an item ID columns based off of the index in the inventory
if(index_patron):
    patron_df = pd.merge(patron_df, inventory_df.reset_index())
    patron_df = patron_df.drop(['Title', 'Author_First', 'Author_Last', 'Num_Checkouts'], axis = 1)
    patron_df = patron_df.rename(columns = {'index': 'Item_ID'})

# Export the DFs as CSV

In [19]:
patron_df.to_csv('Clean_Patron.csv')
inventory_df.to_csv('Clean_Inventory.csv')

# Final State of Dataframes

In [20]:
patron_df.sample(10)

,Patron_ID,Item_ID
67545,15307,2311
33477,13458,62
55746,13408,742
7632,11254,6079
64304,13606,1531
34381,13831,856
10009,11483,1416
28520,15520,435
50554,18038,1040
48462,12885,9961


In [21]:
inventory_df.sample(10)

,Title,Author_First,Author_Last,Num_Checkouts
11183,the louisiana purchase,rebecca,rowell,1
8673,dangerous alterations,elizabeth lynn,casey,2
11170,shattered mirror,amelia,atwaterrhodes,1
189,snap,margaret and ayliffe,mayo,35
4878,cape refuge,terri,blackstock,4
8930,molly the true story of the amazing dog who r...,colin,butcher,2
5918,100 perfect hair days stepbysteps for pretty ...,jenny,strebe,4
5996,from 1 to 10,richard,scarry,4
10529,pleasures of the cottage garden,rand b,lee,2
2697,the little cowboy and the big cowboy,margaret,hillert,8


# Exploring the Data

In [22]:
# Number of entries in patron
len(patron_df)

74729

In [23]:
# Number of unique patrons
patron_df['Patron_ID'].nunique()

1015

In [24]:
# Number of unique items in the inventory
len(inventory_df)

11438

In [25]:
# Number of unique titles
inventory_df['Title'].nunique()

11230